In [30]:
import warnings 
warnings.filterwarnings('ignore')

In [31]:
from crewai import Agent,Task,Crew

In [32]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

In [33]:
openai_api_key = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serper_api_key

In [35]:
from crewai_tools import (
  ScrapeWebsiteTool,
  PDFSearchTool,
  SerperDevTool,
    FileReadTool
)


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
semantic_search_resume = PDFSearchTool()
read_resume = FileReadTool(file_path="./rohit.pdf",name="PDF Resume Reader",description="Read pdf resume that can be used to optimize it")

In [36]:
read_resume

FileReadTool(name='PDF Resume Reader', description="PDF Resume Reader() - A tool that can be used to read ./rohit.pdf's content.", args_schema=<class 'crewai_tools.tools.file_read_tool.file_read_tool.FixedFileReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000002BA7186D580>, file_path='./rohit.pdf')

In [37]:
from crewai import Agent,Task,Crew

#Agent 1
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [52]:
#Agent 2

profiler = Agent(
    role="Skills Manager",
    goal="Do increditble research on job applicants "
         "to help them stand out in the given job posting",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess,you can add and remove skills which is requiredin the job posting you dissect "
        "and synthesize information "
        "you can analyse the skills in the job posting and in user resume"
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [39]:
#Agent 3

resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [40]:
#Agent 4

resume_formatter = Agent(
    role="Latex Writter",
    goal="Latex code writter ",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "As an expert LaTeX code writer, you excel in crafting precise, error-free, and syntactically correct LaTeX documents. Your proficiency ensures the selection of appropriate font sizes and colors, creating well-structured and visually appealing documents that adhere to best practices."    )
)

In [41]:
#Agent 5
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [42]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) or Job description ({job_discription}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [43]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, tech stack , contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [44]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the short summary, work experience, skills, "
        "and education and if possible achivement. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [45]:
resume_formatter_task = Task(
    description = (
 """
To create a professional and visually appealing resume using LaTeX, follow the provided guidelines carefully. Here’s a detailed breakdown of how to structure and style your resume:

1. **Template Selection:**
   - Use the best resume template to structure the content.

2. **Formatting Links:**
   - Underline all hyperlinks.

3. **Experience Section:**
   - Bold the job titles.
   - Place the corresponding dates next to the job titles in **month/year** format, also in bold.

4. **Topic and Subtopic Styling:**
   - Make the main topics (e.g., Projects, Experience, Achievements, Education) sky blue and underline them.
   - Bold the main topics and subtopics, with subtopics in a slightly less bold weight.
   - Always place sub-headings below the main headings.

5. **Font and Layout:**
   - Use an appropriate font size to fit the resume content on a single page.
   - Cover the entire horizontal space with some margins to ensure the content is well-balanced and readable.
"""),expected_output="update the resume effectively and good format in one page",
    context = [resume_strategy_task],
    agent=resume_formatter,
    output_file="updated_resume.md",
)

In [46]:
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [47]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            resume_formatter,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           resume_formatter_task,
           interview_preparation_task],

    verbose=True
)

2024-05-22 06:07:09,051 - 2060 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [48]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/rohitw3code',
    'job_discription':'',
    'personal_writeup': """
    I am Rohit 4th year b.tech AI student from sarala birla university
    """
}

In [49]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) or Job description () to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/rohitw3code) URLs, and personal write-up (
    I am Rohit 4th year b.tech AI student from sarala birla university
    ). Utilize tools to extract and synthesize information from these sources.
 [DEBUG]: == [Personal Profiler for Engineers] Task output: 


 [DEBUG]: == Working Agent: Resume Strategist for Engineers
 [INFO]: == Starting Task: Using the profile and job requiremen

Action Input: {"search_query": "Rohit Kumar b.tech AI student sarala birla university"} 


Search results: Title: Rohit Kumar - Laper | LinkedIn
Link: https://in.linkedin.com/in/rohit-kumar-66522518a
Snippet: I am Rohit Kumar, a passionate Data Scientist with a strong background in Artificial… | Learn more about Rohit Kumar's work experience, education, ...
---
Title: Rohit Kumar Mahto - Sarala Birla University - SBU, Ranchi - LinkedIn
Link: https://in.linkedin.com/in/rohit-kumar-mahto-7b0482259
Snippet: Rohit Kumar Mahto. Student at Sarala Birla University - SBU, Ranchi. Sarala Birla University - SBU, Ranchi. Ranchi, Jharkhand, India.
---
Title: Sarala Birla University: Courses, Fees, Admission 2024, Placements ...
Link: https://www.shiksha.com/university/sarala-birla-university-ranchi-61731
Snippet: The University is approved by UGC. The University offers courses such as BTech, BBA, BCA, BA (English-Hons), and Bachelor of Yoga and Naturopathy at UG levels.
---
Title: Faculty List And

Final Answer:
Rohit Kumar
rohitcode005@gmail.com
linkedin - github.com/Rohitw3code - kaggle

EDUCATION
Sarala Birla University
Ranchi, Jharkhand
B.Tech in AI (Artificial Intelligence)
11/2021 - 04/2025

TECHNICAL SKILLS
Programming Languages: Python, C+, C, Java, JavaScript, C sharp
Libraries and Tools: Tensorflow, HuggingFace, Langchain, PyTorch, Sklearn, Pandas, Numpy, OpenCV, LLM, flask, react, NLP, nodejs, android app, SQL, ETL, IOT, esp32, arduino, Git, firebase, MongoDB, Linux, Neural network, CUDA, no SQL, Docker
ML Architectures: CNN, YOLO, Transformers (BERT, LSTM)

WORK EXPERIENCE
Data Engineer
Remote-internship at Taiyo.ai, San Francisco, California, USA
06/2022 - 09/2022
- Conducted web scraping to extract data from both the World Bank and IADB websites
- Python scripting in OOP (Object-Oriented Programming) and establishment of a robust data pipeline for seamless integration with Google Cloud
- Data Preprocessing, data analysis using Pandas and NumPy, Matplotlib, Seaborn



I need to gather all relevant information from Rohit Kumar's resume to create tailored interview questions and talking points based on his experience and skills.

Action: PDF Resume Reader
Action Input: {"pdf": "./rohit.pdf"} 

I encountered an error while trying to use the tool. This was the error: 'charmap' codec can't decode byte 0x90 in position 539: character maps to <undefined>.
 Tool PDF Resume Reader accepts these inputs: PDF Resume Reader() - A tool that can be used to read ./rohit.pdf's content.

Thought: I need to gather all relevant information from Rohit Kumar's resume to create tailored interview questions and talking points based on his experience and skills.

Action: Search a PDF's content
Action Input: {"query": "Rohit Kumar", "pdf": "./rohit.pdf"} 

Relevant Content:
Rohit Kumar - rohitcode005@gmail.com - linkedin - github.com/Rohitw3code - kaggle EDUCATION Sarala Birla University Ranchi ,Jharkhand B.Tech in AI (Artificial Intelligence) 11/2021 - 04/2025 TECHNICAL SKI

In [50]:
from IPython.display import Markdown, display
display(Markdown("./updated_resume.md"))

\documentclass[10pt,a4paper]{article}

\usepackage[margin=1in]{geometry}
\usepackage{helvet}
\renewcommand{\familydefault}{\sfdefault}

\usepackage{xcolor}
\definecolor{skyblue}{RGB}{135,206,235}

\begin{document}

\begin{center}
\textbf{\large Rohit Kumar} \\
rohitcode005@gmail.com | linkedin - github.com/Rohitw3code - kaggle
\end{center}

\section*{EDUCATION}
Sarala Birla University \\
Ranchi, Jharkhand \\
\textbf{B.Tech in AI (Artificial Intelligence)} \\
\textbf{11/2021 - 04/2025}

\section*{TECHNICAL SKILLS}
\textbf{Programming Languages:} Python, C+, C, Java, JavaScript, C sharp \\
\textbf{Libraries and Tools:} Tensorflow, HuggingFace, Langchain, PyTorch, Sklearn, Pandas, Numpy, OpenCV, LLM, flask, react, NLP, nodejs, android app, SQL, ETL, IOT, esp32, arduino, Git, firebase, MongoDB, Linux, Neural network, CUDA, no SQL, Docker \\
\textbf{ML Architectures:} CNN, YOLO, Transformers (BERT, LSTM)

\section*{WORK EXPERIENCE}
\textbf{Data Engineer} \\
Remote-internship at Taiyo.ai, San Francisco, California, USA \\
\textbf{06/2022 - 09/2022}
\begin{itemize}
  \item Conducted web scraping to extract data from both the World Bank and IADB websites
  \item Python scripting in OOP (Object-Oriented Programming) and establishment of a robust data pipeline for seamless integration with Google Cloud
  \item Data Preprocessing, data analysis using Pandas and NumPy, Matplotlib, Seaborn
\end{itemize}

\textbf{Prompt Engineer} \\
Scale \\
Developed a community app for developers (8k+ downloads) \\
Codeddit App - Playstore

\section*{EXTRACURRICULAR ACTIVITIES}
\textbf{Samsung Solve for Tomorrow CSR} \\
FITT, Delhi \\
\textbf{06/2023}
\begin{itemize}
  \item Achieved a remarkable ranking in the Top 30 among more than 70,000 participants for developing an innovative mobile application that provides instantaneous solutions and support to a developer
  \item Laper App - Playstore
\end{itemize}

\textbf{Ranked Top 5 in Anveshana} \\
Synopsys, Delhi \\
\textbf{02/2024}
\begin{itemize}
  \item Developed a live IoT project utilizing ESP Cam, Arduino, and machine learning for train accident prevention
  \item Implemented obstacle detection and drowsiness detection for the train driver, resulting in winning cash prizes and goodies in a competition
\end{itemize}

\end{document}